# Neural Network creation

This script takes results from "aggregated results", and generates neural
networks based on them.


In [ ]:
import sys
from utils.setup import SetupNeuralNetworks

argv           = sys.argv[1:]
# argv           = ['-c', 'cfg_single_lon120.yml']
argv = ["-c", "cfg_testing.yml"]

setup = SetupNeuralNetworks(argv)


# Create Neural Networks

Rasp et al.'s CBRAIN `fc_model` is used here for reference. As we are
doing a simpler version, I've opted to create our own, shorter function
for NN creation.

## List of models

In [ ]:
from neural_networks.models import generate_models
model_descriptions = generate_models(setup)

## Read aggregated results and generate CausalSingleNN models

Currently, results are loaded from a previously created file. This was
made for testing purposes, and we may want to replace it for an analysis.

In that replacement, the idea would be to execute more or less the same
as aggregate_results up to the moment the file is currently saved, and
continue with that object (aggregated results) here.

## Generate SingleNN models

## Generate CausalSingleNN models

# Training

## Data generator

In [ ]:
from neural_networks.data_generator import build_train_generator, build_valid_generator

## Fit models

Train all models in the model list and store the results.

In [ ]:
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from neural_networks.cbrain.learning_rate_schedule import LRUpdate
from neural_networks.cbrain.save_weights import save_norm

for model_description in model_descriptions:
    print(model_description)
    
    input_vars_dict = model_description.input_vars_dict
    output_vars_dict = model_description.output_vars_dict
    
    with build_train_generator(
        input_vars_dict, output_vars_dict, setup
    ) as train_gen, build_valid_generator(
        input_vars_dict, output_vars_dict, setup
    ) as valid_gen:
        lrs = LearningRateScheduler(LRUpdate(
            init_lr = setup.init_lr,
            step = setup.step_lr,
            divide = setup.divide_lr
        ))
        tensorboard = tf.keras.callbacks.TensorBoard(
            log_dir=Path(
                model_description.get_path(setup.tensorboard_folder),
                model_description.get_filename()
            ),
            histogram_freq=0,
            write_graph=True,
            write_images=False,
            update_freq="epoch",
            profile_batch=2,
            embeddings_freq=0,
            embeddings_metadata=None,
        )
        early_stop = EarlyStopping(
            monitor="val_loss",
            patience=setup.train_patience
        )
        model_description.fit_model(
            x = train_gen,
            validation_data = valid_gen,
            epochs = setup.epochs,
            callbacks = [lrs, tensorboard, early_stop],
            verbose = setup.train_verbose,
        )
        model_description.save_model(setup.nn_output_path)
        # Doing this after saving the model avoids having to create
        # the folder ourserlves
        save_norm( 
            input_transform = train_gen.input_transform,
            output_transform = train_gen.output_transform, 
            save_dir = str(model_description.get_path(setup.nn_output_path)),
            filename = model_description.get_filename()
        )

